In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [0]:
df = spark.read.format("delta")\
          .option("header",True)\
          .option("inferSchema",True)\
          .load("abfss://bronze@netflixdatastorage.dfs.core.windows.net/netflix_titles") 

In [0]:
df.printSchema()

root
 |-- duration_minutes: string (nullable = true)
 |-- duration_seasons: string (nullable = true)
 |-- type: string (nullable = true)
 |-- title: string (nullable = true)
 |-- date_added: string (nullable = true)
 |-- release_year: string (nullable = true)
 |-- rating: string (nullable = true)
 |-- description: string (nullable = true)
 |-- show_id: string (nullable = true)
 |-- _rescued_data: string (nullable = true)



In [0]:
df = df.fillna({"duration_minutes":0,"duration_seasons":1})

In [0]:
df.withColumn("type_flag",when(col("type")=="Movie",1)).withColumn("type_flag",when(col("type")=="TV Show",2))

DataFrame[duration_minutes: string, duration_seasons: string, type: string, title: string, date_added: string, release_year: string, rating: string, description: string, show_id: string, _rescued_data: string, type_flag: int]

In [0]:
from pyspark.sql.window import Window

In [0]:
df = df.withColumn("duration_ranking",dense_rank().over(Window.orderBy(col("duration_minutes"))))

/databricks/python/lib/python3.11/site-packages/pyspark/sql/connect/expressions.py:1017: UserWarning: WARN WindowExpression: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
  warnings.warn(


In [0]:
df.createOrReplaceTempView("temp_view")

/databricks/python/lib/python3.11/site-packages/pyspark/sql/connect/expressions.py:1017: UserWarning: WARN WindowExpression: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
  warnings.warn(


In [0]:
df = spark.sql('''
               select * from temp_view
               ''')

In [0]:
df_vis = df.groupBy("type").agg(count("*").alias("count"))
display(df_vis)

type,count
null,1
TV Show,1969
Movie,4265
1944,1


Databricks visualization. Run in Databricks to view.

In [0]:
df.write.format("delta")\
    .mode("overwrite")\
    .option("path","abfss://silver@netflixdatastorage.dfs.core.windows.net/netflix_titles")\
    .save() 